In [159]:
project_name = 'inno_hack'

isInColab = True

if (isInColab):
    from google.colab import drive
    drive.mount('/colab')

Drive already mounted at /colab; to attempt to forcibly remount, call drive.mount("/colab", force_remount=True).


In [160]:
import numpy as np
import pandas as pd
import sklearn
from importlib import reload
import sys
from os import path

sys.path.append('/colab/MyDrive/colab/shared/custom_modules')
import my_sklearn_transformers as trsfm
reload(trsfm)

pd.set_option('display.max_columns', 500)
path_to_data_dir = lambda x: path.join('/colab/MyDrive/colab/data/{}'.format(project_name), *(x if isinstance(x, str) is False else [x]))

In [161]:
U = pd.read_csv(path_to_data_dir('company_and_its_investor.csv'))

In [162]:
U.head(3)

,Unnamed: 0,inn__investor,name__investor,type__investor,startup_stage__investor,market__investor,services__investor,technologies__investor,type_of_ownership__investor,investment_round__investor,investition_from_dol__investor,investition_to_dol__investor,tech_focus__investor,fund_total_rub__investor,fund_total_dol__investor,num_of_investments__investor,num_of_exits__investor,geography__investor,study_format__investor,num_of_people_in_company_from__investor,num_of_people_in_company_to__investor,num_of_participants__investor,okved_main__investor,okved_secondary__investor,corp_stage__investor,business_model__investor,monetary_support__investor,inn__company,name__company,got_support_from__company,did_get_support__company,service__company,foundation_date__company,tech_focus__company,stage_of_development__company,market__company,technology__company,business_model__company,main_okved__company,okved_secondary__company,msp_category__company,is_export__company,inno_cluster_member__company,skolcovo_member__company,is_inno_company__company,is_startup__company,current_profit__company,current_profit_tax__company,current_revenue__company
0,0,7703638456,da vinci capital,VentureFund,"['ранний рост', 'расширение']","['cybersecurity', 'transport & logistics', 'bu...",['инвестиции'],"['ar/vr', 'big data', 'блокчейн', 'интернет ве...",частный,"['раунд а', 'раунд в', 'раунд с+']",1000000.0,50000000.0,[],25000.0,397.0,17.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7704794345,"ооо ""геттакси рус""",карта инновационных решений,да,добавлено - карта инновационных решений,2011-11-03,['транспорт и логистика'],расширение,['transport & logistics'],[],['b2c'],62.09,"['82.99', '46.19', '63.11.1', '47.91.2', '69.2...",NaN,нет,да,нет,да,нет,-23.0,-15.0,1507.0
1,1,7728461088,new industry ventures,VentureFund,"['ранний рост', 'расширение', 'посевная']",['cleantech'],['инвестиции'],"['3d моделирование', 'big data', 'интернет вещ...",частный,"['seed', 'раунд а', 'раунд в']",500000.0,3000000.0,[],4000.0,62.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7714762021,"ооо ""скайер ит""",московский акселератор,нет,московский акселератор,2008-12-23,['недвижимость и строительство'],ранний рост,['proptech'],['беспилотники'],['b2b'],72.19,"['62.09', '85.42', '62.02', '63.11.1', '30.30....",юл микро,нет,да,да,да,нет,-16450.0,0.0,19127.0
2,2,7728461088,new industry ventures,VentureFund,"['ранний рост', 'расширение', 'посевная']",['cleantech'],['инвестиции'],"['3d моделирование', 'big data', 'интернет вещ...",частный,"['seed', 'раунд а', 'раунд в']",500000.0,3000000.0,[],4000.0,62.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7714762021,"ооо ""радуга""",карта инновационных решений,да,добавлено - карта инновационных решений,2008-12-23,['недвижимость и строительство'],ранний рост,['proptech'],['беспилотники'],['b2b'],72.19,"['62.09', '85.42', '62.02', '63.11.1', '30.30....",юл микро,нет,да,да,да,нет,-16450.0,0.0,19127.0


In [163]:
import ast
def try_load_list(val):
    try:
        res = ast.literal_eval(val)
        if isinstance(res, list):
            return res
        else:
            return []
    except Exception as e:
        return []

In [164]:
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer

onehot_company = ['stage_of_development__company', 'main_okved__company', 'msp_category__company', 'is_export__company', 'inno_cluster_member__company', 'skolcovo_member__company',
                  'is_inno_company__company', 'is_startup__company']
multilabel_company = ['tech_focus__company', 'market__company', 'technology__company', 'business_model__company', 'okved_secondary__company']


num_investor = ['investition_from_dol__investor', 'investition_to_dol__investor', 'fund_total_rub__investor', 
                'fund_total_dol__investor', 'num_of_investments__investor', 'num_of_exits__investor']
onehot_investor = ['type_of_ownership__investor']
multilabel_investor = ['startup_stage__investor', 'market__investor', 'services__investor', 'technologies__investor', 'investment_round__investor', 'tech_focus__investor']

def map_lists(x):
    return np.array([try_load_list(i) for i in x], dtype=object)

preprocessor_X = make_union(
    make_pipeline(
        trsfm.DfSelector(onehot_company),
        SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=''),
        OneHotEncoder(sparse=False, handle_unknown='ignore')
    ),
    *([make_pipeline(
        trsfm.DfSelector([col]),
        FunctionTransformer(np.squeeze),
        FunctionTransformer(map_lists),
        # FunctionTransformer(print),
        trsfm.MultilabelEncoder()
    ) for col in multilabel_company])
)

preprocessor_Y = make_union(
    make_pipeline(
        trsfm.DfSelector(onehot_investor),
        SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=''),
        OneHotEncoder(sparse=False, handle_unknown='ignore')
    ),
    make_pipeline(
        trsfm.DfSelector(num_investor),
        SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0),
        MinMaxScaler()
    ),
    *([make_pipeline(
        trsfm.DfSelector([col]),
        FunctionTransformer(np.squeeze),
        FunctionTransformer(map_lists),
        trsfm.MultilabelEncoder()
    ) for col in multilabel_investor])
)

In [175]:
sklearn.__version__

'0.22.2.post1'

In [165]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1)
for train, test in sss.split(U, U['type_of_ownership__investor'].fillna('')):
    train_idx, test_idx = train, test

In [166]:
X_tr = preprocessor_X.fit_transform(U.iloc[train_idx])
X_te = preprocessor_X.transform(U.iloc[test_idx])
y_tr = preprocessor_Y.fit_transform(U.iloc[train_idx])
y_te = preprocessor_Y.transform(U.iloc[test_idx])

WARN: MultilabelEncoder takes only one col at a time! Given  (351, 0)
WARN: MultilabelEncoder takes only one col at a time! Given  (40, 0)


In [167]:
from joblib import dump

dump(preprocessor_X, path_to_data_dir('fund_classifier_preprocessor_X.joblib'))
dump(preprocessor_Y, path_to_data_dir('fund_classifier_preprocessor_Y.joblib'))

['/colab/MyDrive/colab/data/inno_hack/fund_classifier_preprocessor_Y.joblib']

In [168]:
unique_fund = U.drop_duplicates(subset=['name__investor'])
investors_space = preprocessor_Y.transform(unique_fund)
investors_space_name = unique_fund['name__investor'].values
fund_name_te = U.iloc[test_idx]['name__investor'].values

WARN: MultilabelEncoder takes only one col at a time! Given  (58, 0)


In [169]:
investors_space.shape

(58, 75)

In [170]:
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape, y_tr.min(), y_tr.max()

((351, 585), (40, 585), (351, 75), (40, 75), 0.0, 1.0)

# Модель

Генерация предсказания построена так:
Данные о компании, для которой нужно получить рекомендацию, переводяться в n-мерный вектор, который при помощи 2-слойной модели проецируется в m-мерное пространство сервисов. Далее сравнивается L2 норм расстояния от проекции вектора компании до векторов сервисов, и по данным расстояниям сервисы сортируются в порядке убывания релевантности.  

In [171]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.losses import MeanSquaredError

es_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.001, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

loss = MeanSquaredError()

l_size = 256
dropo = 0.5

model = tf.keras.Sequential(layers=[
    layers.Dense(l_size, input_shape=(X_tr.shape[1],), activation='sigmoid'),
    layers.Dropout(dropo),
    layers.Dense(y_tr.shape[1], activation='sigmoid')
])

model.compile(optimizer='adam', loss=loss)

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               150016    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 75)                19275     
Total params: 169,291
Trainable params: 169,291
Non-trainable params: 0
_________________________________________________________________


In [172]:
model.fit(X_tr, y_tr, validation_data=(X_te, y_te), epochs=3000, callbacks=[es_cb])

Epoch 1/3000
11/11 [==============================] - 0s 13ms/step - loss: 0.2362 - val_loss: 0.1839
Epoch 2/3000
11/11 [==============================] - 0s 5ms/step - loss: 0.1828 - val_loss: 0.1742
Epoch 3/3000
11/11 [==============================] - 0s 5ms/step - loss: 0.1741 - val_loss: 0.1712
Epoch 4/3000
11/11 [==============================] - 0s 6ms/step - loss: 0.1714 - val_loss: 0.1684
Epoch 5/3000
11/11 [==============================] - 0s 4ms/step - loss: 0.1694 - val_loss: 0.1651
Epoch 6/3000
11/11 [==============================] - 0s 5ms/step - loss: 0.1657 - val_loss: 0.1638
Epoch 7/3000
11/11 [==============================] - 0s 5ms/step - loss: 0.1647 - val_loss: 0.1618
Epoch 8/3000
11/11 [==============================] - 0s 5ms/step - loss: 0.1626 - val_loss: 0.1599
Epoch 9/3000
11/11 [==============================] - 0s 5ms/step - loss: 0.1595 - val_loss: 0.1576
Epoch 10/3000
11/11 [==============================] - 0s 5ms/step - loss: 0.1575 - val_loss: 0.155

In [174]:
model.save(path_to_data_dir('model_fund_classifier.h5'))

In [173]:
from numpy import linalg as LA

preds = model.predict(X_te)
for i in range(len(preds)):
    idx_sorted = np.argsort(np.apply_along_axis(LA.norm, 1, investors_space - preds[i], ord=2))
    print("Pred={}, y={}".format(investors_space_name[idx_sorted[:3]], fund_name_te[i]))

Pred=['winter capital' 'dv capital' 'cabra vc'], y=winter capital
Pred=['фрии' 'the untitled ventures' 'leta capital'], y=фрии
Pred=['pulsar vc' 'the untitled ventures' 'yellowrockets'], y=фрии
Pred=['digital evolution ventures' 'i2bf global ventures' 'венчурный фонд нти'], y=i2bf global ventures
Pred=['фрии' 'da vinci capital' 'leta capital'], y=роснано
Pred=['фрии' 'pulsar vc' 'the untitled ventures'], y=фрии
Pred=['фрии' 'the untitled ventures' 'leta capital'], y=фрии
Pred=['moscow seed fund' 'rbf ventures' 'dv capital'], y=фонд поддержки проектов нти
Pred=['moscow seed fund' 'rbf ventures'
 'дальневосточный фонд высоких технологий'], y=terravc
Pred=['фрии' 'yellowrockets' 'the untitled ventures'], y=фрии
Pred=['венчурный фонд мтс' 'np capital' 'the untitled ventures'], y=венчурный фонд мтс
Pred=['pulsar vc' 'np capital' 'the untitled ventures'], y=pulsar vc
Pred=['flashpoint vc' 'larix' 'addventure'], y=российский фонд прямых инвестиций (рфпи)
Pred=['фрии' 'yellowrockets' 'the unti

In [111]:
from joblib import load

def predict(company, services, answer_in):
    company = company.rename(columns=lambda x: '{}__company'.format(x))
    services = services.rename(columns=lambda x: '{}__investor'.format(x))
    
    X = load(path_to_data_dir('fund_classifier_preprocessor_X.joblib')).transform(np.array([company]))
    services_space = load(path_to_data_dir('fund_classifier_preprocessor_Y.joblib')).transform(services)

    proj = tf.keras.models.load_model('fund_classifier_model.h5').predict(X)[0]

    idx_sorted = np.argsort(np.apply_along_axis(LA.norm, 1, services_space - proj, ord=2))

    return answer_in[idx_sorted]

array([[0.6020387 , 0.6090586 , 0.64530996, 0.16150413, 0.18848638,
        0.13797712, 0.1210919 , 0.69529539, 0.99617955, 0.63596809,
        0.79630776, 0.2804911 , 0.36472287, 0.63168706, 0.56508606,
        0.85026221, 0.16214328, 0.29812185, 0.82219159, 0.74422255,
        0.82991964, 0.62508803, 0.6090724 , 0.27044022, 0.33888006,
        0.49355758, 0.32844409, 0.88587455, 0.93177516, 0.7587735 ,
        0.11129599, 0.79302351, 0.11047949, 0.01111092, 0.95154676,
        0.74462908, 0.0369213 , 0.37356102, 0.84699442, 0.28378096,
        0.29967524, 0.93411645, 0.74429194, 0.59064975, 0.01435412,
        0.84537683, 0.90280191, 0.30739076, 0.65577362, 0.6830999 ,
        0.06828213, 0.6853836 , 0.48883236, 0.80428016, 0.87811654,
        0.68576405, 0.03649095, 0.3222333 , 0.74138292, 0.46384741,
        0.47549574, 0.57094435, 0.27928267, 0.79134884, 0.60981802,
        0.85486535, 0.06300596, 0.32130265, 0.92132523, 0.26858342,
        0.38137524, 0.1914696 , 0.36244921, 0.49

In [120]:
def train(train_data, val_data):
    pre_X = load(path_to_data_dir('fund_classifier_preprocessor_X.joblib'))
    pre_Y = load(path_to_data_dir('fund_classifier_preprocessor_Y.joblib')).transform(train_data)

    X_tr = pre_X.transform(train_data)
    y_tr = pre_Y.transform(train_data)

    X_te = pre_X.transform(val_data)
    y_te = pre_Y.transform(val_data)

    es_cb = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0.001, patience=10, verbose=0,
        mode='auto', baseline=None, restore_best_weights=True
    )

    model = tf.keras.models.load_model(path_to_data_dir('fund_classifier_model.h5'))
    
    model.fit(X_tr, y_tr, validation_data=(X_te, y_te), epochs=100, callbacks=[es_cb])

    model.save(path_to_data_dir('fund_classifier_model.h5'))

19